In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
    !pip3 install pickle5 -q
    !apt-get install p7zip-full -q
    #!7z x 'drive/MyDrive/Colab Notebooks/Project/FreiHAND_split.7z'
    !7z x 'drive/MyDrive/Colab Notebooks/Project/RHD_published_v2.zip'
    !cp 'drive/MyDrive/Colab Notebooks/Project/Networks.py' .
    !cp 'drive/MyDrive/Colab Notebooks/Project/HandDataset.py' .
    !cp 'drive/MyDrive/Colab Notebooks/Project/Rotations.py' .
    !cp 'drive/MyDrive/Colab Notebooks/Project/hand3d/data/BinaryDbReader.py' .

    PATH = 'drive/MyDrive/Colab Notebooks/Project/'
except:
    IN_COLAB = False
    PATH = ''

In [4]:
if IN_COLAB:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)

In [13]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from importlib import reload
import torch.nn.functional as F

from HandDataset_noCoords import *
from Networks import *

manualSeed = 42
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [2]:
batch_size = 512
img_size = 64
# nc, nz = 3, 128 # number of channels in training images
# ngf, ndf = 64, 64
# num_epochs = 30
# lr, beta1 = 0.0002, 0.5 # hyperparam for Adam optimizers
# lmbda = 10

In [14]:
real_transform = transforms.Compose([
    transforms.CenterCrop(112),
    transforms.Resize(img_size),
    transforms.ToTensor(),
    #transforms.Normalize((.5,.5,.5),(.5,.5,.5)) # (replace with actual mean and std)
])

real_ds = HandDataset(
    img_dir='../../../Glasgow/FreiHAND_split/together/rgb',
    mask_dir='../../../Glasgow/FreiHAND_split/together/mask',
    anno_path='../../../Glasgow/FreiHAND_split/FreiHAND_anno_training.pickle',
    black_size=224,
    zfill=8,
    ext='.jpg',
    transform=real_transform,
    masked=True,
    centered=False,
    coef = 2.2,
    normalise=False
)

In [12]:
# synth_transform = transforms.Compose([
#     transforms.Resize(img_size),
#     transforms.ToTensor(),
#     #transforms.Normalize((.5,.5,.5),(.5,.5,.5)) # (replace with actual mean and std)
# ])

# synth_ds = HandDataset(
#     img_dir='RHD_published_v2/training/color',
#     mask_dir='RHD_published_v2/training/mask',
#     anno_path='RHD_published_v2/training/anno_training.pickle',
#     black_size=320,
#     zfill=5,
#     ext='.png',
#     transform=synth_transform,
#     masked=True,
#     centered=True,
#     coef = 2.2,
#     normalise=True
# )

In [19]:
data_loader = DataLoader(real_ds, batch_size=batch_size, shuffle=False, drop_last=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [21]:
from __future__ import print_function, unicode_literals

import pickle#5 as pickle
import os
import scipy.misc
import struct
import numpy as np

# chose if you want to create a binary for training or evaluation set
# set = 'training'
set = 'training'
branch = 'freihand'

### No more changes below this line ###


# function to write the binary file
def write_to_binary(file_handle, image, mask, kp_coord_xyz, kp_coord_uv, kp_visible):
    """" Writes records to an open binary file. """
    bytes_written = 0
    # 1. write kp_coord_xyz
    for i in range(22*3):
        file_handle.write(struct.pack('f',0.0))

    # 2. write kp_coord_uv
    for coord in range(21*2):
        file_handle.write(struct.pack('f',0.0))

    # 3. write camera intrinsic matrix
    # for K_row in K_mat:
        # for K_element in K_row:
            # file_handle.write(struct.pack('f', K_element))

    # to make it divisible by 4
    file_handle.write(struct.pack('B', 255))
    file_handle.write(struct.pack('B', 255))
    file_handle.write(struct.pack('B', 255))

    # 4. write image
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            file_handle.write(struct.pack('B', image[x, y, 0]))
            file_handle.write(struct.pack('B', image[x, y, 1]))
            file_handle.write(struct.pack('B', image[x, y, 2]))

    # 5. write mask
    for x in range(mask.shape[0]):
        for y in range(mask.shape[1]):
            file_handle.write(struct.pack('B', mask[x, y]))

    # 6. write visibility
    for x in range(21):
        file_handle.write(struct.pack('B', 255))

# binary file we will write
file_name_out = PATH+'hand3d/data/bin/%s_%s.bin' % (branch,set)

In [44]:
print('Starting conversion process...')

with open(file_name_out, 'wb') as fo:
    for i,batch in enumerate(data_loader):
        img,_,mask,_,_ = batch
        #img = img.to(device)
        #mu, logVar = enc(img)
        #z = reparameterize(mu, logVar)
        #out = dec(z)

        img_norm = (((img-img.min()) / (img.max()-img.min())) * 255.).permute(0,2,3,1).type(torch.uint8).cpu()
        #out_norm = (((out-out.min()) / (out.max()-out.min())) * 255.).permute(0,2,3,1).type(torch.uint8).cpu()
        mask_norm = (((mask-mask.min()) / (mask.max()-mask.min())) * 255.).permute(0,2,3,1).type(torch.uint8)
        #xyz = xyz.numpy().astype(np.float32)
        #uv = uv.numpy().astype(np.float32)
        #vis = vis.numpy().astype(np.uint8)

        for j in range(img.shape[0]):
            write_to_binary(fo, img_norm[j], mask_norm[j].squeeze(), -1, -1, -1)

        #if (i % 10) == 0:
        print('%d / %d batches done: %.3f percent' % (i, len(data_loader), i*100.0/len(data_loader)))

Starting conversion process...
0 / 64 batches done: 0.000 percent
1 / 64 batches done: 1.562 percent
2 / 64 batches done: 3.125 percent
3 / 64 batches done: 4.688 percent
4 / 64 batches done: 6.250 percent
5 / 64 batches done: 7.812 percent
6 / 64 batches done: 9.375 percent
7 / 64 batches done: 10.938 percent
8 / 64 batches done: 12.500 percent
9 / 64 batches done: 14.062 percent
10 / 64 batches done: 15.625 percent
11 / 64 batches done: 17.188 percent
12 / 64 batches done: 18.750 percent
13 / 64 batches done: 20.312 percent
14 / 64 batches done: 21.875 percent
15 / 64 batches done: 23.438 percent
16 / 64 batches done: 25.000 percent
17 / 64 batches done: 26.562 percent
18 / 64 batches done: 28.125 percent
19 / 64 batches done: 29.688 percent
20 / 64 batches done: 31.250 percent
21 / 64 batches done: 32.812 percent
22 / 64 batches done: 34.375 percent
23 / 64 batches done: 35.938 percent
24 / 64 batches done: 37.500 percent
25 / 64 batches done: 39.062 percent
26 / 64 batches done: 40

In [1]:
!pip install scipy==1.2.0 -q
%cd 'drive/MyDrive/Colab Notebooks/Project/hand3d/'
%tensorflow_version 1.x

/content/drive/MyDrive/Colab Notebooks/Project/hand3d
TensorFlow 1.x selected.


In [34]:
%run training_posenet.py

/device:GPU:0
enter modetraining
enter filevaegan_training.bin
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pi

<Figure size 432x288 with 0 Axes>

In [ ]:
%run eval2d_gt_cropped.py

use retrained? 0
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
reader read

In [ ]:
%run eval2d_gt_cropped.py

use retrained? 1
which version? rhd_raw
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.da

In [ ]:
%run eval2d_gt_cropped.py

use retrained? 1
which version? rhd
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` 

In [ ]:
%run eval2d_gt_cropped.py

use retrained? 1
which version? cyclegan_disc
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the 

In [ ]:
%run eval2d_gt_cropped.py

use retrained? 1
which version? cyclegan_auth
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the 

In [2]:
%run eval2d_gt_cropped.py

use retrained? 1
which version? vaegan
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.dat